Estudiaremos en el presente Notebook las cotizaciones y las distintas carácterísticas que componen los ETFs de BlackRock, conocidos como iShares.
Todos ellos serán los cotizados en **USD-$**, para poder realizar comparativas y agrupaciones de forma más sencilla.

Debido a esta condición, añadida a un mínimo de 20 años desde su salida al mercado para tener un número considerable de registros, nos encontramos ante **87 ETFs** distintos para analizar.


# Data Project

## Dataset

Nos quedará un Dataset compuesto por el histórico de la capitalización de cada ETF de iShare, que añadiremos a nuestro entorno de desarrollo en la carpeta de *pre-processed*, donde incluiremos todos los csv.

Estos csv han sido generados despúes de un primer paso de darle calidad a nuestros datos a partir de un script de Python que transforma los datos de la carpeta *raw* a los csv de *pre-processed*, preparándolos para su uso en KDB+.

En nuestro mismo directorio, recibiremos otro csv, donde estará la información a cargar para la tabla Resume, también generado por Python.


  > Tabla **Historical**: Registra el histórico de cotización de los ETFs. 
  >  - ***date***: La fecha del registro del histórico.    
  > - ***nav_value***: El valor de la participación en esa fecha.    
  > - ***shares***: El nº de participaciones en el mercado del ETF en esa fecha.  
  > - ***ticker***: EL ticker identificativo del ETF.

  > Tabla **Resume**: Se trata de la información general del los ETFs de iShare. 
  >  - ***ticker***: El identificativo de cada ETF de iShares.    
  > - ***inception_date***: La fecha a la que sale al mercado el ETF.    
  > - ***net_assets***: El total de capitalización del ETF en el momento de sacar los datos.  
  > - ***equity_class***: Muestra el tipo de activo que compone el ETF. *Large/Mid Cap - All Cap - Large Cap - Mid Cap - Small Cap*
  > - ***region***: Muestra la región de dónde son las empresas. *Asia Pacific - North America  - Europe - Latin America - Global - Middle East and Africa*
  > - ***market***: Muestra el tipo de mercado que tiene las empresas. *Developed - Emerging*

1. Crearemos en un primer momento las tablas vacías.
2. Importaremos desde los csv la información y la insertaremos en nuestras tablas.

In [ ]:
meta historical:([]date:`date$();nav_value:`float$();shares:`long$();ticker:`$())
meta resume_etf:([ticker:`$()]inception_date:`date$();net_assets:`float$();equity_class:`$();region:`$();market:`$())

//@[`historical;`ticker;`g#];

## Data Processing - Filter and Quality
Nuestros datos están disponibles en la carpeta *Data*, pero se presentan en un formato de Excel y con información poco valiosa y muy dispersa para el estudio. Por tanto consideramos que se encuentran en un proceso raw en nuestro proyecto de Ingeniería de Datos.

Por ello contamos con 2 scripts a realizar, donde filtraremos, renombraremos y daremos calidad a los datos, para posteriormente poder estudiarlos de mejor manera.

Pero antes de ello realizaremos la instalación de las librerías correspondientes, actualizadas en el *requirements.txt*:

In [ ]:
system "pip install -r requirements.txt"

In [ ]:
system "python3 Data/script_etf_a_csv.py Data/raw-excels/ Data/iShares-UnitedStates2.xlsx"
system "python3 Data/script_resume_etf.py Data/iShares-UnitedStates2.xlsx"

## Data Ingest

Procedemos a importar los archivos de los ETFs que se encuentran en la ruta : *Data/pre-processed/etf/* y los cargaremos  para poder operar con ellos.

In [ ]:
csv_files: system "ls Data/pre-processed/etf/*.csv";

Insertamos en nuestra tabla **'historical'**  usando un iterador y las características de programación funcional y su compilación de derecha a izquierda que nos aporta el lenguaje.

Importamos los datos del csv, actualizamos la columna ticker con el nombre que queremos y realizamos un upsert sobre la tabla.

In [ ]:
//Ingesta de HISTORICAL
{ ticker_name: ssr[ssr[; "Data/pre-processed/etf/"; ""] x; ".csv"; ""]; 
    `historical upsert update ticker:`$ticker_name from ("DFJ"; enlist csv) 0: `$x;
 } each csv_files;
//Ingesta de RESUME
`resume_etf insert ("SDFSSS"; enlist csv) 0: `$"Data/pre-processed/resume.csv";

In [ ]:
show historical
show resume_etf

# New Data and Metadata

Procederemos a crear columnas en nuestras tablas con intención de recopilar información sobre el histórico y poder analizar posteriormente con mayor información y datos.

## Daily Returns:
 Añadiremos la diferencia de precios diaria en la tabla de *Historical* para poder calcular la volatilidad total 

In [ ]:
historical: `ticker`date xasc historical;
historical: update daily_retns: (nav_value-prev nav_value)%prev nav_value by ticker from historical
show historical 

## Volatility
Añadimos la volatilidad total por ticker a la tabla *Resume*.

Tomaremos la fórmula de la Volatilidad cómo: Volatility = σ√T, donde T es el nº de periodos a estudiar.

En mercados financieros los años se tratan de 252 días de mercados abiertos, por tanto:

 - Volatilidad Anual: 252 días
 - Volatilidad 6 meses: 126 días
 - Volatilidad 3 meses: 63 días

### Historical Volatility - 1 Year

In [ ]:
year_vol: select vol1y: dev daily_retns * sqrt[252] by ticker from historical where date > max date - 251

### Historical Volatility - 6 Months

In [ ]:
sixm_vol: select  vol6m: var daily_retns * sqrt 126 by ticker from historical where date > max date - 125

### Historical Volatility - 3 Months

In [ ]:
threem_vol: select vol3m: var daily_retns * sqrt 63 by ticker from historical where date > max date - 62

In [ ]:
meta resume_etf: ((resume_etf lj `ticker xkey year_vol) lj `ticker xkey sixm_vol) lj `ticker xkey threem_vol

## Moving Averages

Calcularemos las medias móviles de ciertos periodos:
 - 200 días
 - 200 meses


### SMA 200 Days

In [ ]:
historical: `ticker`date xdesc historical;
historical: update d200sma:mavg[200;nav_value] by ticker from historical

### SMA 200 Months

Usamos la ordenación descendente + first en el fby para coger el primer dato del mes (el último al estar ordenado descendentemente).

In [ ]:
historical: `ticker`date xdesc historical
show historical: update m200sma:mavg[200;nav_value] by ticker from historical where date = (first;date)fby `month$date

In [ ]:
//Vemos los días del mes que son seleccionados para calcular la sma mensual
select from historical where ticker = `EWA, m200sma <> 0N

## Period Performance
Calcularemos los rendimientos del etf en los siguientes periodos, en un valor porcentual(%):
- 5 años
- 1 año
- 6 meses

### Performance - 5 Years

In [ ]:
perf_5y: select perf_5y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date=max date) or (date=max date-5*365)

### Performance - 3 Years

In [ ]:
perf_3y: select perf_3y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date- 3*365 - 1)

### Performance - 1 Year

In [ ]:
perf_1y: select perf_1y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date- 1*365)

In [ ]:
meta resume_etf: ((resume_etf lj `ticker xkey perf_5y) lj `ticker xkey perf_3y) lj `ticker xkey perf_1y

## Drawdowns

Calcularemos:
- 5 años
- 3 años
- 1 año

### Drawdown - 5 Years

In [ ]:
drawdown_5y: select drawdown_5y: 100*(max nav_value - min nav_value)%min nav_value by ticker 
    from select from historical where date >= max date - 5 * 365i

### Drawdown - 3 Years

In [ ]:
//Problema - no consigo filtrar los históricos por ticker con distintas fecha --> reestudiarlo

In [ ]:
//Idea inicial sin contar que el min tiene que ser posterior al max
show drawdown_3y: select drawdown_3y: 100*(max nav_value - min_nav_value)%min_nav_value by ticker 
    from select from historical where date >= max date - 3 * 365i

In [ ]:
//Intentos de calcular las fechas de los max y a partir de ahi filtrar para poder coger los min posteriores 
show max_nav_value: select date by ticker from historical where date >= max date - 3 * 365i, nav_value = (max;nav_value) fby ticker

In [ ]:
show min_nav_value_postmax: select from historical where date > (select date from max_nav_value)

### Drawdown - 1 Year

In [ ]:
drawdown_1y: select drawdown_1y: 100*(max nav_value - min nav_value)%min nav_value by ticker 
    from select from historical where date >= max date - 1 * 365i


[0;31mhistorical[0m: [0;31mhistorical[0m

In [ ]:
meta resume_etf: ((resume_etf lj `ticker xkey drawdown_5y)lj `ticker xkey drawdown_3y) lj `ticker xkey drawdown_1y

## Inflows

Calculamos las entradas de dinero en el etf, con la ampliación/salida de dinero en el mercado del etf por la variación mensual de las participaciones en circulación.

In [ ]:
historical: `ticker`date xasc historical

In [ ]:
historical: update minflow: 100*(shares - prev shares)% prev shares by ticker from historical where date = (last;date)fby `month$date

# ETF CONDITIONS

La idea es analizar los datos recopilados y poder acotar bajo que *parámetros/características* nuestros etf se comportan de cierto modo. De esta manera, podremos hacer predicciones con mayores garantías.

In [ ]:
// A realizar

# Extras + info futuro


In [ ]:
count select ticker from resume_etf where market = `$"Developed"
select from historical where ticker = `EWA, m200sma <> 0N
count select ticker from resume_etf where market = `Emerging

In [ ]:
//Filtrado  mezclando las dos tablas
select from historical where ticker in exec ticker from resume_etf where market = `$"Emerging"
//Lo mismo
show a: select ticker from resume_etf where market = `$"Developed"
select from historical where ticker in a[`ticker]

In [ ]:
//
ES UN EJEMPLO DE COMO FUNCIONAN LAS VISTAS EN KDB+/Q --> CREAR FUNCIONES !!

tabla: ([] ticker: ("AAPL";"GOOGL";"MSFT"); price: (100;200;150))

// Definir función vista
vista_filtrada: {[tabla] select from tabla where price > 150}

// Llamar a la función vista
vista_filtrada tabla

//
//
// Calcular la volatilidad para cada símbolo en la tabla historical
historical_volatility:select vol: sqrt var daily_retns by ticker from historical
resume_etf: historical_volatility lj `ticker xkey resume_etf
//

In [ ]:
select distinct equity_class from resume_etf
select distinct region from resume_etf

In [ ]:
//maximo aumento
select from historical where daily_retns=(max; daily_retns) fby ticker